<a href="https://colab.research.google.com/github/SARA3SAEED/LLM-2/blob/main/17Day_3_Foundation_keyword_semantic_bm25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import math
from scipy import spatial


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install sentence_transformers openai unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_ncc

Let's discuss why we need chunking/ need to divide data - context length

In [3]:
txt = '''I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought'''

In [ ]:
len(txt)

6422

In [4]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
from transformers import BertTokenizer, BertModel


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
# text = "This is a long text that needs to be encoded and decoded."
encoded_text = tokenizer.encode(txt, padding=True, truncation=True, return_tensors='pt')

In [6]:
encoded_text.shape

torch.Size([1, 512])

In [7]:
decoded_text = tokenizer.decode(encoded_text[0], skip_special_tokens=True)

In [8]:
decoded_text

'i would like to get your all thoughts on the bond yield increase this week. i am not worried about the market downturn but the sudden increase in yields. on 2 / 16 the 10 year bonds yields increased by almost 9 percent and on 2 / 19 the yield increased by almost 5 percent. key points from the cnbc article : * * * the “ taper tantrum ” in 2013 was a sudden spike in treasury yields due to market panic after the federal reserve announced that it would begin tapering its quantitative easing program. * * * * * major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. * * * * * however, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation. * * the recent rise in bond yields and u. s. inflation expectations has some investors wary that a re

In [9]:
len(decoded_text)

2542

Let's build basic search using TF-IDF

In [10]:

# List of documents
documents = [
    "The cat is playing in the garden",
    "A dog and cat are good pets",
    "Cats love to chase mice",
    "machine learning is based on algorithms ",
    "Deep learning uses neural networks",
    "Recurrent networks have connections"
]

# Tokenize documents into words

tokenized_docs = [word_tokenize(d) for d in documents]

# Inverted index mapping words to document IDs
index = {}
for i, doc in enumerate(tokenized_docs):
    for word in doc:
        if word not in index:
            index[word] = []
        index[word].append(i)

print(f'Inverted Index:\n {index}')



Inverted Index:
 {'The': [0], 'cat': [0, 1], 'is': [0, 3], 'playing': [0], 'in': [0], 'the': [0], 'garden': [0], 'A': [1], 'dog': [1], 'and': [1], 'are': [1], 'good': [1], 'pets': [1], 'Cats': [2], 'love': [2], 'to': [2], 'chase': [2], 'mice': [2], 'machine': [3], 'learning': [3, 4], 'based': [3], 'on': [3], 'algorithms': [3], 'Deep': [4], 'uses': [4], 'neural': [4], 'networks': [4, 5], 'Recurrent': [5], 'have': [5], 'connections': [5]}


In [11]:

# Compute TF-IDF weights for words
def tfidf(word, id):
    # Term frequency - no. of times word appears in document
    tf = tokenized_docs[id].count(word)

    # No. of documents containing the word
    df = len(index[word]) if word in index else 0

    # Inverse document frequency
    idf =  math.log(len(documents) / (df+1))

    return tf * idf



In [12]:
# Search query
query = "machine learning"
query_words = word_tokenize(query)

In [13]:
scores = {id:0 for id in range(len(documents))}
for q in query_words:
    if q in index:
        for id in index[q]:
            scores[id] += tfidf(q, id)

scores

{0: 0, 1: 0, 2: 0, 3: 1.791759469228055, 4: 0.6931471805599453, 5: 0}

In [14]:

# Rank documents
ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Return ranked documents
for id, score in ranked:
    print(f"Document {id}: {documents[id]}")
    print(f"Score: {score}")

Document 3: machine learning is based on algorithms 
Score: 1.791759469228055
Document 4: Deep learning uses neural networks
Score: 0.6931471805599453
Document 0: The cat is playing in the garden
Score: 0
Document 1: A dog and cat are good pets
Score: 0
Document 2: Cats love to chase mice
Score: 0
Document 5: Recurrent networks have connections
Score: 0


In [15]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pandas as pd


In [16]:
documents = [
    "The cat is playing in the garden",
    "A dog and cat are good pets",
    "Cats love to chase mice",
    "Machine learning is based on algorithms",
    "Deep learning uses neural networks",
    "Recurrent networks have connections"
]


In [17]:
model = SentenceTransformer('all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [19]:
embeddings = model.encode(documents,convert_to_tensor=True)

In [20]:
embeddings.shape

torch.Size([6, 384])

In [21]:
embeddings[0]

tensor([ 9.8462e-02, -4.5544e-02,  9.6758e-03, -5.4596e-02, -4.5926e-02,
         5.0185e-02, -6.7055e-03, -7.2435e-03,  1.1958e-02,  4.6411e-02,
        -1.3074e-02, -1.6105e-02,  8.1282e-03, -1.6081e-03,  4.6857e-02,
         1.8405e-03, -2.9331e-02,  4.3547e-02,  2.5231e-02,  6.9381e-03,
         5.1756e-03,  6.1812e-02,  2.7769e-02, -4.5394e-02, -2.6461e-02,
         4.6035e-02, -3.5700e-02,  9.1378e-03, -6.8735e-02, -1.6847e-02,
        -1.9701e-02,  4.8861e-02, -3.4722e-02,  6.5693e-02, -6.6862e-02,
        -2.4257e-02, -1.6287e-02, -7.9458e-02,  7.8634e-02,  3.6495e-02,
        -8.9438e-02, -1.5068e-02,  1.4100e-02, -4.4663e-02, -1.0404e-02,
         5.1208e-02,  5.1050e-02, -4.7541e-02,  4.9426e-02, -1.4960e-01,
        -1.8853e-02,  8.2226e-02, -7.8429e-03, -4.6720e-02, -6.0168e-02,
        -2.6692e-02,  5.4851e-02,  2.5637e-02, -9.7486e-03, -3.0156e-02,
        -1.2376e-02, -1.7353e-02,  3.7652e-02,  1.0458e-01,  1.8895e-02,
        -1.0423e-01, -4.3090e-02,  2.5660e-02,  5.6

In [22]:
# User query
query = "large language models"

# Encode query
query_embedding = model.encode(query)

In [23]:
scores = [spatial.distance.cosine(query_embedding, doc) for doc in embeddings]


In [24]:
scores

[1.0273892376571894,
 0.911145806312561,
 0.974956814199686,
 0.7824788987636566,
 0.7213239073753357,
 0.5819475054740906]

In [25]:
scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in embeddings]

# Print each score
for i, doc in enumerate(documents):
    print(f"Document {i}: {documents[i]}")
    print(f"Score: {1 - scores[i]:.4f}")

print("Most similar document:", documents[scores.index(min(scores))])

Document 0: The cat is playing in the garden
Score: -0.0274
Document 1: A dog and cat are good pets
Score: 0.0889
Document 2: Cats love to chase mice
Score: 0.0250
Document 3: Machine learning is based on algorithms
Score: 0.2175
Document 4: Deep learning uses neural networks
Score: 0.2787
Document 5: Recurrent networks have connections
Score: 0.4181
Most similar document: Recurrent networks have connections


In [27]:
def semantic_search(query,embeddings, documents):
    query_embedding = model.encode(query)

    document_embeddings = embeddings
    scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in document_embeddings]

    for i, doc in enumerate(documents):
        print(f"{doc}: {1- scores[i]:.4f}")

    most_similar_doc = documents[scores.index(min(scores))]
    print("Most similar document:", most_similar_doc)

In [26]:
def semantic_search(query,embeddings, documents):
    query_embedding = model.encode(query)

    document_embeddings = embeddings
    scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in document_embeddings]

    # Print scores
    for i, doc in enumerate(documents):
        print(f"{doc}: {1 - scores[i]:.4f}")

    most_similar_doc = documents[scores.index(min(scores))]
    print("Most similar document:", most_similar_doc)


In [29]:
semantic_search("machine learning", embeddings, documents)

The cat is playing in the garden: -0.0488
A dog and cat are good pets: 0.0513
Cats love to chase mice: 0.0171
Machine learning is based on algorithms: 0.6723
Deep learning uses neural networks: 0.4380
Recurrent networks have connections: 0.3609
Most similar document: Machine learning is based on algorithms


In [28]:
semantic_search("machine learning",embeddings,documents)

The cat is playing in the garden: -0.0488
A dog and cat are good pets: 0.0513
Cats love to chase mice: 0.0171
Machine learning is based on algorithms: 0.6723
Deep learning uses neural networks: 0.4380
Recurrent networks have connections: 0.3609
Most similar document: Machine learning is based on algorithms


#Let's try bm25

## BM25 vs TF-IDF: A Comparison

Both BM25 (Best Matching 25) and TF-IDF (Term Frequency-Inverse Document Frequency) are fundamental techniques used in information retrieval tasks like search engines and document ranking. They both aim to identify documents most relevant to a user's query by analyzing the importance of terms within documents and the document collection. However, they differ in their approaches:

**Focus:**

* **TF-IDF:** Primarily focuses on the **statistical importance** of terms. It emphasizes terms that appear frequently within a document (high TF) but are rare across the document collection (low IDF).
* **BM25:** Considers both **statistical importance** and **document characteristics**. It builds upon TF-IDF by incorporating document length and a saturation factor to refine the weighting of terms.

**Strengths:**

* **TF-IDF:** Simple and computationally efficient. Effective for identifying documents containing the exact query terms.
* **BM25:** Generally considered more effective than TF-IDF, particularly for short queries or retrieving documents with less frequent terms. It addresses the limitations of TF-IDF by:
    * **Accounting for document length:** Short documents containing the query term might be more relevant than longer documents with the same term. BM25 assigns higher weight to terms in shorter documents.
    * **Addressing term frequency saturation:**  In TF-IDF, terms appearing very frequently within a document (extremely high TF) can dominate the score. BM25 introduces a saturation factor (k1) to limit the impact of excessively frequent terms.

**Weaknesses:**

* **TF-IDF:** Can be susceptible to documents "keyword stuffing" common terms to appear more relevant. It doesn't account for term proximity or synonyms.
* **BM25:** Requires tuning parameters (k1 and b) that can affect performance. It's slightly more complex to implement compared to TF-IDF.

**Choosing Between BM25 and TF-IDF:**

* **TF-IDF:** A good starting point for simpler retrieval tasks or when computational efficiency is a priority.
* **BM25:** Often preferred for more sophisticated retrieval systems, especially for short queries or dealing with large document collections.

**In summary:**

Both BM25 and TF-IDF are valuable tools for information retrieval. While TF-IDF offers simplicity and efficiency, BM25 often provides more accurate results by considering additional factors. The choice between them depends on the specific needs of your retrieval system and the characteristics of your document collection.


In [30]:
# List of documents
documents = [
    "The cat is playing in the garden",
    "A dog and cat are good pets",
    "Cats love to chase mice",
    "machine learning is based on algorithms ",
    "Deep learning uses neural networks",
    "Recurrent networks have connections"
]

In the BM25 (Best Matching) information retrieval formula, k1 and b are parameters that influence how terms are weighted during the scoring process. Here's a breakdown of their roles:

* **k1 (saturation parameter):**
    * Controls the influence of document frequency (DF) on the score.
    * Higher k1 values (>1) **increase the impact** of terms that appear less frequently in the document collection (higher weight for less common words).
    * Lower k1 values (<1) **decrease the impact** of DF, making the score rely more on term frequency within the document itself (favoring frequent terms within a document).
    * A common default value is k1 = 1.2, and the provided example uses k1 = 1.5.

* **b (document length parameter):**
    * Adjusts the influence of document length on the score.
    * Higher b values (closer to 1) give **more weight to shorter documents** as they are assumed to be more relevant if they contain the query term.
    * Lower b values (closer to 0) **reduce the impact of document length**, making the score focus more on term frequency within documents of any length.
    * A common default value is b = 0.75, and the provided example uses b = 0.75.

**Choosing k1 and b:**

The optimal values for k1 and b depend on your specific document collection and retrieval task. Experimentation is often needed to find the best configuration. Here are some general guidelines:

* If your documents are generally short and the collection is large, a lower k1 value and a higher b value might be suitable.
* If your documents are lengthy and the collection is smaller, a higher k1 value and a lower b value could be preferable.

**Additional Notes:**

* The BM25 formula also incorporates Inverse Document Frequency (IDF) to downplay the weight of very common words.
* k1 and b are just two of the parameters that can be adjusted in BM25 variations.



In [31]:
import math
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

# List of documents
documents = [
    "The cat is playing in the garden",
    "A dog and cat are good pets",
    "Cats love to chase mice",
    "machine learning is based on algorithms",
    "Deep learning uses neural networks",
    "Recurrent networks have connections"
]

# Tokenize documents
tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]

# Remove stopwords
stop_words = set(stopwords.words('english'))
tokenized_docs = [[word for word in doc if word not in stop_words] for doc in tokenized_docs]

# Create inverted index
inverted_index = {}
for i, doc in enumerate(tokenized_docs):
    for word in doc:
        if word not in inverted_index:
            inverted_index[word] = []
        inverted_index[word].append(i)

# Calculate IDF for each word
idf = {}
N = len(documents)
for word in inverted_index:
    df = len(inverted_index[word])
    idf[word] = math.log((N - df + 0.5) / (df + 0.5))

# Calculate average document length
avg_doc_len = sum(len(doc) for doc in tokenized_docs) / N

# BM25 parameters
k1 = 1.5
b = 0.75

# Function to calculate BM25 score
def bm25(query, document_id):
    score = 0
    doc_len = len(tokenized_docs[document_id])
    for word in query:
        if word in inverted_index:
            tf = tokenized_docs[document_id].count(word)
            score += idf[word] * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (doc_len / avg_doc_len))))
    return score

# Example usage
query = "machine learning"
query_tokens = word_tokenize(query.lower())
query_tokens = [word for word in query_tokens if word not in stop_words]

scores = [bm25(query_tokens, i) for i in range(N)]
ranked_docs = sorted(range(N), key=lambda i: scores[i], reverse=True)

print("Query:",

      )
print("Ranked documents:")
for i in ranked_docs:
    print(f"Document {i+1}: {documents[i]}")
    print(f"Score: {scores[i]}")
    print()

Query:
Ranked documents:
Document 4: machine learning is based on algorithms
Score: 1.8508572250637418

Document 5: Deep learning uses neural networks
Score: 0.516982535095554

Document 1: The cat is playing in the garden
Score: 0.0

Document 2: A dog and cat are good pets
Score: 0.0

Document 3: Cats love to chase mice
Score: 0.0

Document 6: Recurrent networks have connections
Score: 0.0



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [32]:
import math
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

# List of documents
documents = [
    "The cat is playing in the garden",
    "A dog and cat are good pets",
    "Cats love to chase mice",
    "machine learning is based on algorithms",
    "Deep learning uses neural networks",
    "Recurrent networks have connections"
]

# Tokenize documents
tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]

# Remove stopwords
stop_words = set(stopwords.words('english'))
tokenized_docs = [[word for word in doc if word not in stop_words] for doc in tokenized_docs]

# Create inverted index
inverted_index = {}
for i, doc in enumerate(tokenized_docs):
    for word in doc:
        if word not in inverted_index:
            inverted_index[word] = []
        inverted_index[word].append(i)

# Calculate IDF for each word
idf = {}
N = len(documents)
for word in inverted_index:
    df = len(inverted_index[word])
    idf[word] = math.log((N - df + 0.5) / (df + 0.5))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
avg_doc_len = sum(len(doc) for doc in tokenized_docs)/N

k1 = 1.5
b= 0.75

def bm25(query, document_id):
  score =0
  doc_len = len(tokenized_docs[document_id])
  for word in query:
    if word in inverted_index:
      tf = tokenized_docs[document_id].count(word)
      score += idf[word] * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (doc_len / avg_doc_len))))
  return score

In [38]:
# Example usage
query = "machine learning"
query_tokens = word_tokenize(query.lower())
query_tokens = [word for word in query_tokens if word not in stop_words]

scores = [bm25(query_tokens, i) for i in range(N)]
ranked_docs = sorted(range(N), key=lambda i: scores[i], reverse=True)

print("Query:",

      )
print("Ranked documents:")
for i in ranked_docs:
    print(f"Document {i+1}: {documents[i]}")
    print(f"Score: {scores[i]}")
    print()

Query:
Ranked documents:
Document 4: machine learning is based on algorithms
Score: 1.8508572250637418

Document 5: Deep learning uses neural networks
Score: 0.516982535095554

Document 1: The cat is playing in the garden
Score: 0.0

Document 2: A dog and cat are good pets
Score: 0.0

Document 3: Cats love to chase mice
Score: 0.0

Document 6: Recurrent networks have connections
Score: 0.0



In [ ]:
query = "large language model"
query_tokens = word_tokenize(query.lower())
query_tokens = [word for word in query_tokens if word not in stop_words]

scores = [bm25(query_tokens, i) for i in range(N)]
ranked_docs = sorted(range(N), key=lambda i: scores[i], reverse=True)

print("Query:", query)
print("Ranked documents:")
for i in ranked_docs:
    print(f"Document {i+1}: {documents[i]}")
    print(f"Score: {scores[i]}")
    print()

Query: large language model
Ranked documents:
Document 1: The cat is playing in the garden
Score: 0

Document 2: A dog and cat are good pets
Score: 0

Document 3: Cats love to chase mice
Score: 0

Document 4: machine learning is based on algorithms
Score: 0

Document 5: Deep learning uses neural networks
Score: 0

Document 6: Recurrent networks have connections
Score: 0

